---
# Jupyter notebook tutorial:
#    Fitting models to data with *kafe2*

                                                Johannes Gäßler, October 2022
                                                Günter Quast, April 2020
                                                Cedric Verstege, August 2020
---
## Jupyter Notebook Fundamentals

This file of type `.ipynb` contains a tutorial as a `Jupyter notebook`.
`Jupyter` provides a browser interface with a (simple) development environment for *Python* code
and explanatory texts in intuitive *Markdown* format.
The input of formulas in *LaTeX* format is also supported.

A summary of the most important commands for using *Jupyter* as a working environment can be
found in the notebook
[*JupyterCheatsheet.ipynb*](https://git.scc.kit.edu/yh5078/datenanalyse/-/blob/master/jupyter/JupyterCheatsheet.ipynb)
(German).
Basics for statistical data analysis can be found in the notebooks
[*IntroStatistik.ipynb*](https://git.scc.kit.edu/yh5078/datenanalyse/-/blob/master/jupyter/IntroStatistik.ipynb)
(German) and
[*Fehlerrechnung.ipynb*](https://git.scc.kit.edu/yh5078/datenanalyse/-/blob/master/jupyter/Fehlerrechnung.ipynb) (German).

In *Jupyter*, code and text are entered into individual cells.
Active cells are indicated by a blue bar in the margin.
They can be in two states: in edit mode the input field is white, in command mode it is grayed out.
Clicking in the border area selects the command mode, clicking in the text field of a code cell
switches to edit mode.
The `esc` key can also be used to leave the edit mode.

Pressing `a` in command mode creates a new empty cell above the active cell, `b` creates one below.
Entering `dd` deletes the corresponding cell.

Cells can be either of the type `Markdown` or `Code`.
Entering `m` in command mode sets the type Markdown, entering `y` selects the type Code.

The cell content is processed - i.e. text is formatted or code is executed - by entering `shift+return`,
or `alt+return` if a new, empty cell should also be created.

The settings mentioned here as well as the insertion, deletion or execution of cells can also be
executed via the pull-down menu at the top.

---



# Overview: *kafe*2
***


*kafe2* is a successor to the *kafe* package developed since 2012 for the fitting of model
functions to data.

It supports different data types like simple indexed data,
two-dimensional data points (one quantity *x* and one dependent quantity *y*),
or one-dimensional data that can be binned as a histogram.
Uncertainties of both dependent and independent quantities and, if applicable,
their correlations are supported.
For this purpose, the global covariance matrix is created from different types
of specified uncertainties and taken into account in the fitting.
Compared to many other fitting tools, this possibility is a unique feature of *kafe(2)*.

Simultaneous fitting of several models is also supported.
Each with its own and additionally all or several common parameters
to different data sets ("multi fit").

To minimize the distance between data and model function(s) numerical methods are applied,
which are derived from the open source,
*Python*-based software environment *SciPy* or the *MINUIT* package developed at CERN.
The respective minimized distance metric (or the "cost function") is equal to the negative
natural logarithm of the likelihood function for the data and the given model multiplied by a
factor of two ($-2\,\ln{\cal L}$).
For Gaussian uncertainties of the data points, this corresponds to the method
of least squares (also called "$\chi^2$ method").
Other cost functions based on the method of maximum likelihood are also available for the fitting of probability densities to histograms or indexed data.

To determine the confidence intervals of idividual model parameters as well as
two-dimensional confidence contours for pairs of parameters *kafe2* makes use of the
profile likelihood method.

*kafe2* contains a stand-alone application *kafe2go*, which allows fittings without the creation of own code;
data, model function and options are specified in a *YAML* configuration file.
A fit can then be performed from the command line by calling:

`kafe2go <name>.yaml`

Alternatively *kafe2* can be used as part of a *Python* program.
For the most common use cases *kafe2* has built-in pipelines.
The procedure to use these pipelines is to first call a method that
does the fit, and to then call another method that produces a plot.

**The following examples show the actual procedure.**

#### General settings and useful packages

In [ ]:
from __future__ import division, print_function  # python2-compatibility
import sys, os

# Lines with % or %% at the beginning are so-called "magic commands",
# that specify the cell type or options for displaying graphics.
%matplotlib inline

#### Imports and Presets for *kafe2*:

In [ ]:
import kafe2

import numpy as np, matplotlib.pyplot as plt

# set better default figure size for kafe2
# plt.rcParams['figure.figsize']=[12., 5.]
#        !!!  must be done after importing kafe2 (will else be overwritten)


***
## 1. Simple example for fitting with `kafe2`
***

The following code illustrates the fitting of functions with the *kafe2* fitting tool:

``` python
# Define or read in the data for your fit:
x_data = [1.0, 2.0, 3.0, 4.0]
y_data = [2.3, 4.2, 7.5, 9.4]
# x_data and y_data are combined depending on their order.
# The above translates to the points (1.0, 2.3), (2.0, 4.2), (3.0, 7.5), and (4.0, 9.4).

# Important: Specify uncertainties for the data!
x_error = 0.1
y_error = 0.4

# Pass the information to kafe2:
kafe2.xy_fit("line", x_data, y_data, x_error=x_error, y_error=y_error)
# The string "line" gets mapped to a first degree polynomial for the model function.

# Call another function to create a plot:
kafe2.plot(
    x_label="x",  # x axis label
    y_label="y",  # y axis label
    data_label="Data",  # label of data in legend
)
```

Paste the code into the empty cell below and execute it by pressing `shift+return`.

In [ ]:
# simple example: straight line adjustment with kafe2
# -> insert code here


You should be seeing two plots.
The first one is a regular plot that shows the data, the model, and the fit results.
The second one is a plot showing the so-called profile likelihood of the fit parameters; let's ignore this plot for now.

The plots and the fit results have also been saved to disk.
Try opening the subdirectory `results`.
There should be two png files for the plots, one txt file that contains a human-readable report of the fit results, and one machine-readable yml file that contains the fit results in the YAML format.

### 1.1 Correlated uncertainties
***

To illustrate the possibilities for dealing with uncertainties, we add a further correlated
uncertainty of the dependent quantities *y*:
``` python
kafe2.xy_fit("line", x_data, y_data, x_error=x_error, y_error=y_error, y_error_cor=0.3)
```
Notice how there is an additional keyword argument `y_error_cor` that specified the correlated *y* uncertainty.
Extend the call to `kafe2.xy_fit` in the above code to also specify a correlated *y* uncertainty.
Now repeat the fit.

As expected, such an uncertainty common to all data points does not affect the gradient of the
straight line, but only the parameter *b*, whose uncertainty now becomes greater - corresponding
to the square root of the sum of the squared uncertainties of $\pm 0.58$ from the original fit and the
additional correlated uncertainty of $\pm 0.40$.


In [ ]:
# Copy and extend code from the previous example
# ->


### 1.2 How to find keyword arguments for *kafe2* functions
***

Obviously it is possible to define correlated uncertainties for the `kafe2.xy_fit` function.
But in order to use this functionality you would first need to know that the keyword argument `y_error_cor` exists.
So how could you figure this out?
The first possiblity is to consult the *kafe2* documentation which describes the signatures and arguments of all user-facing functions.
But you can also access this information directly from a Jupyter notebook by accessing the built-in help system.
For example, to get information about the `kafe2.plot` method simply copy the following to the cell below:
``` python
? kafe2.plot
```
The output from the Jupyter help has several parts.
First it prints the signature of the function and what the default values for the arguments are (for `kafe2.plot`the value `None` means that the value should be inferred from the fit).
Then it prints the function docstring which describes the meaning and expected data types of the various arguments.
Finally it prints the location of the function source.

Unfortunately the Jupyter help system does not format function docstrings in the same way that the documentation does so they may be a little hard to read but it's a very quick way of figuring out how a function should be used.
Integrated development environments and advanced text editors (PyCharm, VSCode, Emacs, etc.) usually also have a built-in help system.

In [ ]:
# Just copy "? kafe2.plot" (without quotation marks) here to get help.
# ->


***
## 2. Comparison of Two Different Models
***

On sufficiently small scales every function can be approximated by a line due to Taylor expansion.
However, for many problems this approximation is insufficient.
The following example shows the fitting of a linear and an exponential model to the same data.

To define a model function for *kafe2* simply write it as a *Python* function.
Important: The first argument of the model function is interpreted as the
independent variable of the fit. It is not being changed during the fit and it's the
quantity represented by the *x* axis of the fit.


Definition of two model functions:
``` python
# Our first model is a simple linear function:
def linear_model(x, a, b):
    return a * x + b


# Our second model is a simple exponential function.
# The kwargs in the function header specify parameter defaults.
def exponential_model(x, A_0=1., x_0=5.):
    return A_0 * np.exp(x/x_0)
```

Definition of the data as a *kafe2* `XYContainer`:
``` python
# The data for this exercise:
x_data_2 = [19.8, 3.0, 5.1, 16.1, 8.2, 11.7, 6.2, 10.1]
y_data_2 = [23.2, 3.2, 4.5, 19.9, 7.1, 12.5, 4.5, 7.2]
x_error_2 = 0.3
y_error_rel_2 = 0.15
```
The function `kafe2.xy_fit` needs to be called twice to do two fits:
``` python
results_linear = kafe2.xy_fit(
    linear_model, x_data_2, y_data_2,
    x_error=x_error_2, y_error_rel=y_error_rel_2)
results_exponential = kafe2.xy_fit(
    exponential_model, x_data_2, y_data_2,
    x_error=x_error_2, y_error_rel=y_error_rel_2, profile=True)
```
The fit results are saved to `results_linear` and `results_exponential` (we'll need them later).

Make sure to specify `profile=True` whenever you use a non-linear model function.
A model function is linear if it is a linear function of each of its parameters.
The model function does not need to be a linear function of the independent variable *x*.
Examples: all polynomial model functions are linear, trigonometric functions are non-linear.

If you were to now simply call `kafe2.plot` as you did before it would create a plot of just the exponential fit.
This is because by default `kafe2.plot` creates a plot of the last fit.
To specify that you want a fit of the last two fits, pass `-2` as the first argument:
``` python
kafe2.plot(
    -2,
    # parameter_names=dict(x="t", a=r"\alpha", b=r"\beta", A_0="I_0", x_0="t_0"),
    model_expression=["{a}{x} + {b}", "{A_0} e^{{ {x} / {x_0} }}"]
)
```
When you specify a *Python* function as a model function *kafe2* does not know how this function should be represented with LaTeX.
The keyword argument `model_expression` passes this information to the plot.
Parameter names have to be put between {}.
To get {} for LaTex double them like {{ or }}.

You can uncomment the keyword argument `parameter_names` to change the name of the parameters in the fit.

Paste the code into the empty cell below and execute it by pressing 'shift+return'.

In [ ]:
# Comparison of two models with kafe2
# -> insert code here


You should be seeing three plots.
The first plot is showing the data, the models, and the fit results for both fits.
The other two plots show the profile likelihoods for the linear and the exponential fits.

In the ideal case we only have static Gaussian uncertainties in *y* direction and a linear model function.
The results obtained from the method of least squares ($\chi^2$ method) are then optimal and unbiased.
The conventional "parabolic" parameter errors like $a \pm \Delta a$ are then accurate.

In the aforementioned ideal case the profile likelihoods of single parameters (the "profiles")
(top left and bottom right) are parabolas while the profile likelihoods of two parameters
(bottom left) are paraboloids.
Because a paraboloid is three-dimensional it cannot be directly shown in two dimensions.
Instead the intersects of the profile likelihood with horizontal planes above the cost
function minimum (the "contours") are shown.
In the ideal case the contours are shaped like ellipses.

The profile likelihood of the exponential fit clearly deviates from the shape of a prabola/ellipse -
this is to be expected since the exponential model is not a linear function of its parameter $x_0$.
However, the profile likelihood of the fit using the linear model is also clearly distorted.
The reasons for this are discussed below.

### 2.1 Distortions due to Uncertainties in *x* Direction
***

If errors in *x* direction are used, fitting a straight line also becomes a
non-linear problem.
This is because during the fit *x* errors are converted to *y* errors by multiplying
them with the model function derivative by *x*.
As a consequence the total error becomes a function of the fit parameters in a way
that turns the originally linear regression into a non-linear problem.
To illustrate this, we repeat the same fit as above with increased uncertainties on
the x values:
``` python
kafe2.xy_fit("line", x_data_2, y_data_2, x_error=4*x_error_2, y_error_rel=y_error_rel_2)
kafe2.plot()
```

In [ ]:
# Enter the above code here:
# -->


You should be seeing that the profile likelihood has become more asymmetrical by increasing the *x* errors.
This is because higher values for $a$ result in a higher model function derivative and thus higher total errors which reduces the cost function value.
Conversely lower values for $a$ result in a lower model funtion derivative, lower total errors, and higher cost function values.
While the parameter $b$ is not directly affected by *x* errors it is indirectly being affected due to its negative correlation with $a$.

### 2.2 Distortions due to Relative Uncertainties in *y* Direction
***

Relative *y* errors by default also distort the profile likelihood.
Ideally these *y* errors would be calculated to the true *y* values (which are unknown).
As a stand-in they are calculated as relative to the model.
This causes the *y* errors to become parameter-dependent, which in turn makes the regression problem non-linear.

Alternatively relative *y* errors can be calculated as relative to the data (by setting `errors_rel_to_model=False`).
This keeps the regression linear but it biases the fit results.
This is because data points which randomly fluctuate to smaller values are given a smaller uncertainty than
data points which randomly flucutate to larger values.

The following code can be used to get a comparison between the two modes:
``` python
kafe2.xy_fit("line", x_data_2, y_data_2, x_error=x_error_2, 
             y_error_rel=4*y_error_rel_2, errors_rel_to_model=True)
kafe2.xy_fit("line", x_data_2, y_data_2, x_error=x_error_2, 
             y_error_rel=4*y_error_rel_2, errors_rel_to_model=False)
kafe2.plot(-2)
```
Note: The first case is the default so `errors_rel_to_model=True` can actually be omitted without changing anything.

In [ ]:
# Enter the above code here:
# -->


### 2.3 Output of Fit Results as Variables
***

In many applications it is necessary to continue to use the output of a fit in program code.
This can be done by saving the return value `kafe2.xy_fit` which is a *Python* dictionary with the fit results.
For example, the fit results of the fits with the original *x* errors were saved to `results_linear` and `results_exponential`.

A formatted output (for the linear fit results) can be obtained with the following line:
``` python
print("\n\n".join(f"====== {k} ======\n{v}" for k, v in results_linear.items()))
```
When using *kafe2* objects directly these values can also be obtained by calling the objects' properties.

In [ ]:
# Test output here
# -->


### 2.4 Hypothesis Test to Assess the Models
***

The graphical output does not clearly indicate which of the models is acceptable.
For this purpose a hypothesis test can be performed which indicates the so-called $\chi^2$
probability - i.e. the probability of obtaining a worse value for $\chi^2$ at the
minimum than the observed one.
A higher value corresponds to a better fit.

It is calculated from the cumulative density function of the $\chi^2$ distribution:
``` python
from scipy import stats

def chi2prob(chi2, ndf):
  """ chi2-probability

    Args:
      * chi2: chi2 value
      * ndf: number of degrees of freedom

    Returns:
      * float: chi2 probability
  """

  return 1.- stats.chi2.cdf(chi2, ndf)
```

Enter the code for the $\chi^2$ probability in the cell and check the
two results you got above.

**Hint**: you can either copy the values for $\chi^2$ and the number of degrees of freedom
from the output of the previous cells or you can access them from the fit results
saved to `results_linear` and `results_exponential`
The fit results are dicts and you need to access the properties `goodness_of_fit` and `ndf`.
The property `cost_function_value` is not suitable because it is the sum of $\chi^2$ and
correctional terms.

In [ ]:
# Checking the quality of the fits
# -> enter code here


When using *kafe2* outside this notebook the $\chi^2$ probability can be accessed directly from the property `chi2_probability` of the fit results.

### 2.5 Defining Models via SymPy

If *SymPy* (Symbolic Python) is installed it can be used to define model functions.
For example, the linear and exponential models above can be defined like this:

``` python
linear_model = "linear_model: x a b -> a * x + b"
exponential_model = "exponential_model: x A_0 x_0=5.0 -> A_0 * exp(x / x_0)"
```
The string `->` marks the separation between argument definition and the model function expression.
The beginning of the model definition up to `:` defines the model function name (can be omitted).

The advantage of using the *SymPy* definitions above is that *kafe2* can use them to automatically derive LaTeX representations for the model functions.
Repeat the fit of the two models using the two model definitions above.

In [ ]:
# Fit with SymPy-defined models
# -> enter code here


### 2.6 Influencing the Graphical Output
***

The graphical output was suboptimal in some regards.
For example, two data sets were shown in the legend although both models used the exact same data.
In addition, marker properties and colors can be customized.
However, this is a little more complicated than the e.g. changing the labels.
The `kafe2.plot` function does not have arguments to specify things like colors.
Instead the returned `kafe2.Plot` object (notice the capitalization) has to be used:
``` python
p = kafe2.Plot(-2)
```

To influence the graphic *kafe2* provides a method `Plot.customize` which can be used
to specify values for *matplotlib* parameters for different graphic elements
(*plot_types*: 'data', 'model_line', 'model_error_band', 'ratio', 'ratio_error_band').

The parameters relevant for a *plot_type* and their current values
can be displayed using a function of the *Plot* class:
``` python
p.get_keywords('model_error_band')
```

The names used for objects and possible values correspond to the names in the configuration file
*matplotlibrc* for *matplotlib*.

To for example change the name for the data set and suppress the second output, use the following call:
``` python
p.customize('data', 'label', ["test data", None])
```
The first argument specifies the subplot for which to set keywords.
The second argument specifies which keyword to set.
The third argument is a list of values to set for the keyword for each fit
managed by the plot object.

Alternatively the third argument can be a list of tuples consisting of fit indices
and values.
``` python
p.customize('data', 'label', [(0, "test data"), (1, None)])
```
This syntax makes it possible to set keywords for only a part of the fits managed by the
plot object.
In this particular case the keyword argument `data_label` of `kafe2.plot` could also have been used.

Marker type, size and color of the marker and error bars can also be customized:
``` python
# data
p.customize('data', 'marker', ['o', 'o'])
p.customize('data', 'markersize', [5, 5])
p.customize('data', 'color', [(0, 'blue'), (1,'blue')]) # note: although 2nd label is suppressed
p.customize('data', 'ecolor', [(0, 'blue'), (1, 'blue')]) # note: although 2nd label is suppressed
```

The corresponding values for the model function can also be customized:
``` python
# model
p.customize('model_line', 'color', ['orange', 'lightgreen'])
p.customize('model_error_band', 'label', [(0, r'$\pm 1 \sigma$'), (1, r'$\pm 1 \sigma$')])
p.customize('model_error_band', 'color', [(0, 'orange')])
p.customize('model_error_band', 'color', [(1, 'lightgreen')])
```

It is also possible to change parameters using *matplotlib* functions.
To change the size of the axis labels, use the following calls:
``` python
# Größe der Achsenbeschriftungen
import matplotlib as mpl
mpl.rc('axes', labelsize=20, titlesize=25)
```
Note: the above call sets the *matplotlib* parameters globally.
Plots unrelated to *kafe2* are also being influenced.

With the *kafe2* plot object the matplotlib plot has to be manually created and shown:
``` python
p.plot()
p.show()
```

In [ ]:
# Enter code to test here:
# -->


### Exercise: Better Parameterization for the Exponential Model

The fit results for the exponential model can be improved by a change in parameterization: $f(x; A_0, \lambda) = A_0 e^{\lambda x}$.
Fit the model with the modified parameterization to the data and plot the results together with linear model and the original exponential model.

**Hint**: in *Python* the name `lambda` is reserved for defining so-called anonymous functions.
You therefore have to use a different name in your model function definition.
The name `varlambda` is automatically displayed as the greek letter lambda in *kafe2* but in principle you can use whatever name you want as long as you set `parameter_names` when plotting.


In [ ]:
# Enter your code here:
# -->


You should be seeing that the fit results are essentially unchanged by the change in parameterization.
However, the profile of $\lambda$ should be nearly parabolical unlike the profile of $x_0$.
As a consequence the conventional description of value and standard deviation $\lambda \pm \Delta \lambda$ is much more accurate than the equivalent $x_0 \pm \Delta x_0$.

---
## Interlude: Object-Oriented Programming
---
Until now fits were performed using the functions `kafe2.xy_fit` and `kafe2.plot`. These functions provide
pre-configured pipelines for fitting models to *xy* data and plotting the results. Internally they use
objects (in the programming sense) which represent things like data or fits as a whole. From now on
the examples will use the objects directly. This is slightly more complicated but it also provides
greater flexibility. The general procedure is as follows:

1. A data container object is created which contains data and the associated uncertainties.
2. A fit object is created from a data container object (or raw data) and a model function.
   As with `kafe2.xy_fit` the default model function is a straight line.
3. The method `do_fit` of the fit object is called to perform the numerical minimization of the cost function.
4. The fit results are extracted from the fit object. This can be done by accessing the properties of the fit
   object, by printing a report, or by creating a plot.

The following implementation is roughly equivalent to the very first example of a line fit.
First an `XYContainer` object is created from the data.
Errors are added to the container by calling the method `XYContainer.add_error`.
``` python
xy_data = XYContainer(x_data, y_data)
xy_data.add_error('x', x_error)
xy_data.add_error('y', y_error)
xy_data.label = 'Data'  # How the data is called in plots
```
Next the fit object is created from the data container.
In this case the method `do_fit` is called immediately but we could do other things beforehand like limiting
or constraining parameters or adding more errors.
``` python
line_fit = Fit(data=xy_data)
line_fit.do_fit()  # This will throw a warning if no errors were specified.
```
Finally, extract the fit results. The `Plot` class is the same as from the previous example.
``` python
line_fit.report()  # Prints fit results to console.

plot = Plot(fit_objects=line_fit)  # Create a kafe2 plot object.
plot.x_label = 'x'  # Set x axis label.
plot.y_label = 'y'  # Set y axis label.
plot.plot()  # Do the plot.
plot.show()  # Just a convenience wrapper for matplotlib.pyplot.show() .
```
Note that the fit results are **not** automatically saved to disk.
To save the fit results in this example, add `plot.save()`.

In [ ]:
from kafe2 import XYContainer, Fit, Plot
# Enter the above code here:
# -->


---
## 3. Special Features of Complex (Non-Linear) Models
---

Another example of a non-linear fit is a damped oscillation of a thread pendulum.
The corresponding measurement data are contained in the following code cell:
``` python
# the data
t = [ ... ]
t_errors = 0.05

a = [ ... ]
a_errors = 0.05
```

In [ ]:
from kafe2 import XYContainer, Fit, Plot

# the data:
t = [0.0, 0.5, 1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0, 4.5, 5.0, 5.5, 6.0, 6.5, 7.0, 7.5, 8.0, 8.5, 9.0, 9.5, 10.0,
     10.5, 11.0, 11.5, 12.0, 12.5, 13.0, 13.5, 14.0, 14.5, 15.0, 15.5, 16.0, 16.5, 17.0, 17.5, 18.0, 18.5, 19.0,
     19.5, 20.0, 20.5, 21.0, 21.5,22.0, 22.5, 23.0, 23.5, 24.0, 24.5, 25.0, 25.5, 26.0, 26.5, 27.0, 27.5, 28.0,
     28.5, 29.0, 29.5, 30.0, 30.5, 31.0, 31.5, 32.0, 32.5, 33.0, 33.5, 34.0, 34.5, 35.0, 35.5, 36.0, 36.5, 37.0,
     37.5, 38.0, 38.5, 39.0, 39.5, 40.0, 40.5, 41.0, 41.5, 42.0, 42.5, 43.0, 43.5, 44.0, 44.5, 45.0, 45.5, 46.0,
     46.5, 47.0, 47.5, 48.0, 48.5, 49.0, 49.5, 50.0, 50.5, 51.0, 51.5, 52.0,52.5, 53.0, 53.5, 54.0, 54.5, 55.0,
     55.5, 56.0, 56.5, 57.0, 57.5, 58.0, 58.5, 59.0, 59.5, 60.0]
t_errors = 0.05

a = [ 6.06,  5.17,  3.29,  0.64, -2.26, -4.56, -5.74, -5.58, -4.12, -1.62,
      1.11,  3.56,  5.12,  5.43,  4.41,  2.53, -0.18, -2.78, -4.65, -5.5,
     -5.04, -3.25, -0.75,  1.79,  3.88,  5.31,  5.2,   3.92,  1.74, -0.85,
     -3.13, -4.71, -5.06, -4.26, -2.48, -0.13,  2.19,  4.07,  4.9,   4.64,
      3.16,  1.17, -1.54, -3.26, -4.59, -4.64, -3.69, -1.83,  0.38,  2.76,
      4.16,  4.58,  4.13,  2.45,  0.28, -1.8,  -3.53, -4.43, -4.31, -3.03,
     -1.05,  1.06,  2.79,  3.97,  4.4,   3.37,  1.92, -0.14, -2.29, -3.7,
     -4.28, -3.84, -2.44, -0.59,  1.27,  3.11,  3.9,   4.02,  2.85,  1.21,
     -0.64, -2.51, -3.41, -3.84, -3.34, -1.75, -0.17,  1.85,  3.23,  3.72,
      3.4,   2.54,  0.67, -1.13, -2.8,  -3.77, -3.65, -2.89, -1.43,  0.42,
      2.2,   3.26,  3.42,  3.25,  1.88,  0.33, -1.35, -3.02, -3.41, -3.32,
     -2.2,  -0.77,  0.92,  2.44,  3.31,  3.44,  2.77,  1.25, -0.13, -1.69, -2.78 ]
a_errors = 0.05

The amplitude as a function of time is given by the following model function:
``` python
# Model function for a pendulum as a one-dimensional,
#     damped harmonic oscillator with zero initial speed:
# s = time, a0 = initial_amplitude, l = length of the string,
# r = radius of the steel ball, g = gravitational acceleration, c = damping coefficient.
def damped_harmonic_oscillator(s, a0, l, r, g, c):
  # Effective length of the pendulum = length of the string + radius of the steel ball:
  l_total = l + r
  omega_0 = np.sqrt(g / l_total) # Phase speed of an undamped pendulum.
  omega_d = np.sqrt(omega_0 ** 2 - c ** 2) # Phase speed of a damped pendulum.
  return a0 * np.exp(-c * s) * (np.cos(omega_d * s) + c / omega_d * np.sin(omega_d * s))
```

Data container and fit object are created as usual:
``` python
# Create data container:
data3 = XYContainer(t, a)
data3.add_error(axis='x', err_val=t_errors)
data3.add_error(axis='y', err_val=a_errors)
data3.axis_labels = ('Time t (s)', 'Amplitude A (°)') 

# Create fit object from data and model function:
fit = Fit(data3, damped_harmonic_oscillator)
```

The model contains a number of parameters defined by "auxiliary measurements".
``` python
# Relevant physical magnitudes and their uncertainties:
lm, delta_lm = 10.000, 0.002  # length of the string, l = 10.0 +- 0.002 m
rm, delta_rm = 0.052, 0.001  # radius of the steel ball, r = 0.052 +- 0.001 m
# Amplitude of the steel ball at x=0 in degrees, a0m = 6 +- 1% degrees:
a0m, delta_a0m = 6.0, 0.01  # Note that the uncertainty on a0m is relative to a0m.
```

The fit takes this into account by considering the corresponding parameters both as
parameters of the fit and as additional data points.
In *kafe2* such parameters restricted by measurements are considered with the help of the
method *Fit.add_parameter_constraint()* and their uncertainties are propagated into
the result of the fit:
``` python
# Constrain model parameters to measurements:
fit.add_parameter_constraint(name='l', value=lm, uncertainty=delta_lm)
fit.add_parameter_constraint(name='r', value=rm, uncertainty=delta_rm)
fit.add_parameter_constraint(name='a0', value=a0m, uncertainty=delta_a0m, relative=True)
```

Alternatively, you could set the parameters to constant values with the method
*Fit.fix_parameter()*;
however, the uncertainties on the final result of the fit would then have to be calculated
using classical error propagation.

A problem with  non-linear fits is that there are often secondary minima of the cost
function - convergence to the global minimum is not guaranteed.
It is therefore necessary to select "reasonable" start parameters for the fit.
This is done using the function *Fit.set_parameter_values()*:
``` python
g_initial = 9.81  # Initial guess for g.
fit.set_parameter_values(g=g_initial, a0=a0m, l=lm, r=rm)
```

If the initial values are completely unknown, the fit should be repeated with a wide range
of initial parameters to check that it consistently converges to the same minimum.

Another means of improving convergence is to limit parameters to "reasonable" intervals.
The parameters *a0*, *l*, and *r* are for example positive by definition.
During the fit however they can become negative.
The following code limits them to positive values:
``` python
fit.limit_parameter("a0", lower=1e-6)
fit.limit_parameter("l", lower=1e-6)
fit.limit_parameter("r", lower=1e-6)
```
For technical reasons parameters can only be limited to closed intervals.
As the lower limit a small value close to zero is specified.
Because no upper value is specified the parameter limit is one-sided.
It is also possible to define two-sided parameter limits:
``` python
fit.limit_parameter("g", lower=9.71, upper=9.91)
```
The above limit is based on the assessment that results outside of these limits are
very unlikely.
It's also a good idea to limit parameters based on the physical properties of the system.
For example the model function only yields real solutions for $c < \frac{g}{l + r}$.
This can be considered with the following code:
``` python
c_max = 0.9 * g_initial / (lm + rm)  # A little lower than our best guess for the limit.
fit.limit_parameter("c", lower=1e-6, upper=c_max)
```

After these preparations the fit can be performed as usual.
The following code example also shows how to access the fit results via properties if
they are to be processed further in the program or if a specific output is desired.
``` python
# Perform the fit
fit.do_fit()
# Optional: Print out a report on the fit results on the console.
#fit.report(show_data=False, show_model=False, show_fit_results=True)

# Custom printout of results:
print("cost function at minimum: %.4g " % fit.cost_function_value,
    " number of degrees of freedom:", fit.ndf)
print(" --> probability: %.1f%%" % (fit.chi2_probability * 100))
print("parameter names:\n", fit.parameter_names)
np.set_printoptions(precision=5, suppress=False)
print("prameter values:\n", fit.parameter_values)
print("parameter uncertainties:\n",fit.parameter_errors)
np.set_printoptions(precision=3, suppress=True)
print("correlation matrix:\n", fit.parameter_cor_mat )
      
# Optional: plot the fit results.
plot = Plot(fit)
plot.plot(fit_info=True)
plot.show()
```


In [ ]:
# Enter code here
# -->


---
## 4. Construction of a Covariance Matrix from Individual Uncertainties
---

To be treated:
  - dealing with complex uncertainties

One of the strengths of _kafe2_ is the support for correlated uncertainties.
This refers to contributions to uncertainties that influence some or all values in
the same way - e.g. because they were recorded with the same measuring instrument
that has a systematic uncertainty.
Uncertainties that are shared between groups of measured values are very common.

The following function is used to specify uncertainties:
> `add_error**( [axis], err_val, name=None, correlation=0, relative=False)`  
  Add an uncertainty source to the data container. Returns an error id which
  uniquely identifies the created error source.  
  **Parameters**  
  • axis (str or int) – 'x'/0 or 'y'/1  
  • err_val (float or iterable of float) – pointwise uncertainty/uncertainties for all data points  
  • name (str or None) – unique name for this uncertainty source. If None, the name
    of the error source will be set to a random alphanumeric string.  
  • correlation (float) – correlation coefficient between any two distinct data points  
  • relative (bool) – if True, err_val will be interpreted as a relative uncertainty  
  **Returns** error name  
  **Return type** str  

It belongs to the container class, but can also be called via a fit class.
With this rather simple interface, independent uncertainties as well as
common absolute or relative uncertainties of data points can be specified.
The specified uncertainties are converted into a covariance matrix of the data points.
If the interface is called several times, the resulting covariance matrices are added
together (in accordance with the rules of elementary error propagation).

A very simple example will illustrate this.
We consider the averaging of four values, which were carried out by two groups with different
measuring methods.
Each of the two groups gives two measurements;
in the first group there is an absolute uncertainty common to the two measurements;
the second group indicates a relative uncertainty correlated between its two measurements, e.g.
caused by a scaling error.
Furthermore, the measurements are based on a common (theoretical) assumption which leads to an
absolute uncertainty common to all measurements.

For this simple problem we use the simplest data structure of *kafe2*, the
_IndexedContainer_, to provide the data:
``` python
from kafe2 import IndexedContainer
idx_data = IndexedContainer([5.3, 5.2, 4.7, 4.8])
```
As model we choose a constant function:
``` python
# The very simple "model":
def average (a):
  return a
```

The uncertainties are then stated as follows
                (Note: For an _IndexedContainer_, the _axis_ parameter is not required!):
  1. each measurement has its own independent uncertainty
   `err_stat = idx_data.add_error([.2, .2, .2, .2])`
  2. the uncertainty common to the first two values
   `err_syst12 = idx_data.add_error([.175, .175, 0., 0.], correlation = 1.)`
  3. the relative uncertainty common to the last two values
   `err_syst34 = idx_data.add_error([0., 0., .05, 0.05], correlation = 1., relative=True)`
  4. the uncertainty common to all values
   `err_syst = idx_data.add_error(0.15, correlation = 1.)`

We should also provide suitable labels for the data:
``` python
idx_data.label = 'Test data'
idx_data.axis_labels = [None,'Measured value (a.o.)']
```

The execution of the fit is well known by now:
``` python
# Set up the fit:
ifit = Fit(idx_data, average)
ifit.model_label = 'average value'

# Perform the fit:
ifit.do_fit()
```

The results can of course be obtained with the _report()_ function, if necessary also as a
graphical representation:
``` python
# Report and plot results:
ifit.report()
p=Plot(ifit)
p.plot()
p.show()
```

If a problem contains several contributions to the overall uncertainty, one would
usually like to study the influence of individual components.
For this purpose, one can comfortably work with the functions *disable_error()* and
*enable_error()* and make appropriate fits:
``` python
print("disabling common sysytematic error")
idx_data.disable_error(err_syst)
_ifit = Fit(idx_data, average)
_ifit.do_fit()
_ifit.report()
#      do not forget to switch on again
idx_data.enable_error(err_syst)
```

In [ ]:
# Enter code here
# -->


---
## 5. Application from the Real World: Fit of a Breit-Wigner Resonance
---

To be treated:
  - dealing with complex uncertainties
  - the creation of an appealing graphical output
  - studying the influence of individual error components

Typically, the uncertainties of the measurement data are much more complex than in the examples
discussed so far.
In most cases there are uncertainties in ordinate and abscissa, and in addition to the
independent uncertainties of each data point there are common, correlated uncertainties for all
of them.

With the method *add_error()* or *add_matrix_error()* uncertainties can be specified on the 'x'
and 'y' data, either in the form of independent or correlated, relative or absolute uncertainties
of all or groups of measured values or by specifying the complete covariance or correlation matrix.
All uncertainties specified in this way are included in the global covariance matrix for the fit.

As an example, we consider measurements of a cross section as a function of the energy near a
resonance.
These are combined measurement data from the four experiments at CERN's LEP accelerator, which
were corrected for effects caused by photon radiation:

Measurements of the hadronic cross section $\sigma_{e^+e^- \to {\rm hadrons}}$ as a function of
the centre-of-mass energy $E$.
``` python
## Data:
# Center-of-mass energy E (GeV)
E = [ 88.387, 89.437, 90.223, 91.238, 92.059, 93.004, 93.916 ]
E_errors = [ 0.005, 0.0015, 0.005, 0.003, 0.005, 0.0015, 0.005 ]
ECor_abs = 0.0017  # correlated absolute errors

# hadronic cross section with photonic corrections applied (nb)
sig = [6.803, 13.965, 26.113, 41.364, 27.535, 13.362, 7.302 ]
sig_errors = [ 0.036, 0.013, 0.075, 0.010, 0.088, 0.015, 0.045 ]
sigCor_rel = 0.0007
```

As a model we use a modified Breit-Wigner resonance with a width dependent on the centre-of-mass
energy ("$s$-dependent width", where $s = E_{CM}^2$):
``` python
## Model:
# Breit-Wigner with s-dependent width
def BreitWigner(E, s0 = 41.0, M = 91.2, G = 2.5):
    s = E*E
    Msq = M*M
    Gsq = G*G
    return s0*s*Gsq/((s-Msq)*(s-Msq)+(s*s*Gsq/Msq))
```

The data container with the uncertainties is created as follows:
``` python
BWdata= XYContainer(ECM, sig)
# Add independent errors:
error_name_sig = BWdata.add_error(axis='x', name = 'deltaE', err_val = E_errors )
error_name_E = BWdata.add_error(axis='y', name = 'deltaSig', err_val = sig_errors )
# Add fully correlated, absolute Energy errors:
error_name_ECor = BWdata.add_error(axis='x', name='Ecor',err_val = ECor_abs, correlation = 1.)
# Add fully correlated, relative cross section errors:
error_name_sigCor = BWdata.add_error(axis='y', name='sigCor',
                            err_val = sigCor_rel, correlation = 1., relative=True)
```

Whether the uncertainties are independent or correlated is determined by the parameter
*correlation*;
for independent uncertainties it is zero, for uncertainties common to all data entries it is one.
Values between 0. and 1. are also allowed;
however, in practice the covariance matrix for describing the overall uncertainty is usually
composed of uncorrelated and fully correlated components.
The names given in the *add_error* function allow the individual error components to be accessed
later.

Fit and result output follow the usual procedure:
``` python
BWfit = Fit(BWdata, BreitWigner)
BWfit.do_fit()
BWfit.report(asymmetric_parameter_errors=True)
# Optional: plot the fit results
BWplot = Plot(BWfit)
BWplot.plot(asymmetric_parameter_errors=True)
BWplot.show()
```

**Enhancement of the graphical output**
To ensure that the type of data is clearly described, suitable names should be assigned.
The lines below must be inserted before the *Fit* object is created.
``` python
BWdata.label = 'QED-corrected hadronic cross-sections'
BWdata.axis_labels = ('CM Energy (GeV)', '$\sigma_h$ (nb)' )
```
Alternatively the following lines can be inserted after creating the fit object:
``` python
BWfit.data_container.label = 'QED-corrected hadronic cross-sections'
BWfit.data_container.axis_labels = ('CM Energy (GeV)', r'$\sigma_h$ (nb)')
```

A suitable name for the model should also be set in the legend for the graphical output.
To do this, insert the line below after the *Fit* object has been created:
``` python
BWfit.model_label = 'Beit-Wigner with s-dependent width'
```

If a nicely set expression for the model function is desired, LaTeX names can be set for the
model, the parameters and the model function:
``` python
# Set LaTeX names for printout in info-bo:
BWfit.assign_parameter_latex_names(E='E', s0=r'{\sigma^0}', M=r'{M_Z}', G=r'{\Gamma_Z}')
BWfit.assign_model_function_latex_name(r'\sigma^{\rm ew}_{e^+e^-\to{\rm hadrons}}')
BWfit.assign_model_function_latex_expression(
               r'{s0}\frac{{ {E}^2{G}^2}}{{({E}^2-{M}^2)^2+({E}^4{G}^2/{M}^2)}}')
```
Note: The doubling of the brackets "{" and "}" is necessary because in *kafe2*, similar to the
Python *format* function, they are also used to pass parameters.

The previous example showed how to modify the band showing the model uncertainty:
``` python
BWplot.customize('model_error_band', 'label', [r'$\pm 1\sigma$'])
```
In this example, however, the model uncertainty is extremely small (well below 0.1%) and
therefore not visible in the graph.
You can suppress the output in the legend with the following specification:
``` python
BWplot.customize('model_error_band', 'label', [None])
```
Sometimes the uncertainty band is covered by the line;
in such cases a dashed or dotted line should be used for the model:
``` python
BWplot.customize('model_line', 'linestyle', [':'])
```

The edges of the plot can also be adjusted.
This can be done via the properties *x_range* and *y_range* od the Plot class:
``` python
BWplot.x_range = (88, 94)
BWplot.y_range = (0, 45)
```

Since this is a non-linear fit, the profile likelihood and confidence contours should
still be displayed.
The following line must be inserted before *BWplot.show()*:
``` python
ContoursProfiler(BWfit).plot_profiles_contours_matrix(show_grid_for='contours')
```
!!! Patience: the calculation of the contours is computationally complex and takes a certain
amount of time!


In [ ]:
from kafe2 import ContoursProfiler  # kafe2 object for contour plots

''' the data for the Breit-Wigner example'''
# Center-of-mass energies E (GeV)
ECM = [ 88.387, 89.437, 90.223, 91.238, 92.059, 93.004, 93.916 ]
E_errors = [ 0.005, 0.0015, 0.005, 0.003, 0.005, 0.0015, 0.005 ]
ECor_abs = 0.0017  # correlated absolute errors

# hadronic cross sections with photonic corrections applied (nb)
sig = [6.803, 13.965, 26.113, 41.364, 27.535, 13.362, 7.302 ]
sig_errors = [ 0.036, 0.013, 0.075, 0.010, 0.088, 0.015, 0.045 ]
sigCor_rel = 0.0007


In [ ]:
# enter the code from above here
# -->


**Study of the influence of individual error components**
To investigate the influence of individual error components on the result, individual sources of
uncertainty can be switched off with the method *disable_error()* and a new fit can be performed,
shown here for the correlated uncertainty of the center-of-mass energies:
``` python
print('!!!  disabling error component ', error_name_ECor)
BWfit.disable_error(error_name_ECor)
BWfit.do_fit()
BWfit.report(show_data=False, show_model=False)

# do not forget to switch on again !
print('!!!  re-enabling error component ', error_name_ECor)
BWfit.enable_error(error_name_ECor)

#### fallback option with new fit object
#print('!!!  disabling error component ', error_name_ECor)
#BWdata.disable_error(error_name_ECor)
#_fit = Fit(BWdata, BreitWigner)
#_fit.do_fit()
#_fit.report(show_data=False, show_model=False)
#BWdata.enable_error(error_name_ECor)
```
The result is almost identical to the previous one, only the uncertainty of the mass is now smaller.
This was also to be expected, because a correlated change of all energies should not influence
the width or height of the resonance.

With the method `enable_error(error_name_ECor)` the uncertainty source is reactivated.


In [ ]:
# try it out here
# -->


***
## 6. Fit to Histogram Data
***

In principle, the fit of a distribution density to a frequency distribution can also be
understood as a functional fit.
However, there are some special features that must be taken into account:

- The function value for a bin corresponding to the value of a distribution density (PDF=Particle
  Density Function) corresponds to the integral of the PDF via the bin
- The uncertainty of a bin entry results from the Poisson distribution, which can only be
  approximated by a Gaussian distribution for very large numbers of entries per bin.

Therefore *kafe2* offers a special method to fit a distribution density to histograms, the
classes *HistContainer* to store the histogram data and *HistFit* to perform the fit:
``` python
from kafe2 import HistContainer, HistFit
```

Twice the negative logarithm of the Poisson likelihood is used as the cost function to evaluate
the agreement of the adjusted PDF with the bin entries in the frequency distribution, other
options are configurable.

In this simple example, we use the frequency distribution of Gaussian-distributed random numbers
to which a Gaussian distribution is fitted.
``` python
def normal_distribution_pdf(x, mu, sigma):
  return np.exp(-0.5 * ((x - mu) / sigma) ** 2) / np.sqrt(2.0 * np.pi * sigma** 2)
```

The data is generated randomly from the standard normal distribution:
``` python
# create a random dataset of 100 random values,
#  following a standard normal distribution with mu=0 and sigma=1
data = np.random.normal(loc=0, scale=1, size=100)
```

The data container and the fit object are created in the same way as in the previous examples:
``` python
# Create a histogram from the dataset by specifying the bin range and the number of bins.
# Alternatively the bin edges can be set.
histogram = HistContainer(n_bins=10, bin_range=(-5, 5), fill_data=data)

# create the Fit object by specifying a density function
fit = HistFit(data=histogram, model_function=normal_distribution_pdf)
```

Carrying out the fit and outputting the results are no different from the procedure used in the
previous examples:
``` python
# do the fit
fit.do_fit()

# Optional: print a report to the terminal
fit.report()

# Optional: create a plot and show it
phist = Plot(fit)
phist.plot()
phist.show()
```

At this point we should take another look at the possibilities for customizing the graphical output.
The plot adapter for histograms knows the values *data*, *model* and *model_density* as *plot_type*.
By calling `print(phist.get_keywords(<plot_type>))` the adjustable parameters can be retrieved.
Here is a suggestion for code to customize the graphic output, which must be placed before the
*phist.plot()* command:
``` python
## reprise: plot customization
#    data
phist.customize('data', 'label', ["random Gaussian data"] )
phist.customize('data', 'marker', ['o'])
phist.customize('data', 'markersize', [5])
phist.customize('data', 'color', ['blue'])
phist.customize('data', 'ecolor', ['blue'])
#    model
phist.customize('model_density', 'label', ["Gaussian PDF"])
phist.customize('model_density', 'color', ["black"])
phist.customize('model', 'label', ["entries per bin"])
phist.customize('model', 'facecolor', ["lightgrey"])
```

In [ ]:
# try here
# -->


***
## 7. Likelihood Fits
***

If only few measurements are available it is not possible to obtain a meaningful frequency
distribution because a coarse division into bins would distort the measurements, while a too
fine division would lead to bins with very few or even zero entries.
The method already used above for fitting a distribution density to a frequency distribution is
then not applicable.
In such cases, a direct fit to the unbinned data using the maximum likelihood method is used.
This procedure is implemented in *kafe2*.
For this the appropriate classes must be imported:
``` python
from kafe2.fit import UnbinnedContainer, UnbinnedFit
```

In this example we use 160 individual measurements of the lifetime of muons from cosmic
radiation stopped in a detector.
The frequency distribution is an exponential distribution over flat ground:

``` python
def pdf(t, tau=2.2, fbg=0.1, a=1., b=9.75):
  """
  Probability density function for the decay time of a myon.
  The pdf is normalized to an integral of one for the interval (a, b).
  :param t: decay time
  :param fbg: background
  :param tau: expected mean of the decay time
  :param a: the minimum decay time which can be measured
  :param b: the maximum decay time which can be measured
  :return: probability for decay time x
  """
  pdf1 = np.exp(-t / tau) / tau / (np.exp(-a / tau) - np.exp(-b / tau))
  pdf2 = 1. / (b - a)
  return (1 - fbg) * pdf1 + fbg * pdf2
```

Please note that the frequency distribution for all possible parameter values must be normalized
to one!

There is only one small particularity regarding the fit procedure:
due to the small number of observations, the background portion is subject to a large uncertainty
and can therefore even become negative during the fit.
To avoid this "unphysical" range of the parameter, the previously shown method
`fit.limit_parameter(<name>, lower=<min>, upper=<max>)` can be used.

All further steps in the following sample code are already known:
``` python
data = UnbinnedContainer(dT) # create the kafe data object
data.label = 'lifetime measurements'
data.axis_labels = ('Myon Life Time ' r'$\tau$' ' (µs)','Density' )

# create the fit object and set the pdf for the fit
LLfit = UnbinnedFit(data=data, model_function = pdf)

# assign latex names for model and parameters for nicer display
LLfit.model_label = 'Exponential decay + flat background'
LLfit.assign_parameter_latex_names(t='t', tau=r'\tau', fbg='f', a='a', b='b')
LLfit.assign_model_function_latex_expression("\\frac{{ (1-{fbg}) \, e^{{-{0}/{tau}}}}}"
    "{{{tau}(e^{{-{a}/{tau}}}-e^{{-{b}/{tau}}})}} + \\frac{{ {fbg} }} {{ {b}-{a} }}")

# Fix the parameters a and b ...
a = 1.0
b = 11.5
LLfit.fix_parameter("a", a)
LLfit.fix_parameter("b", b)
# ... and limit parameter fbg
LLfit.limit_parameter("fbg", lower=0., upper=1.)

LLfit.do_fit()  # perform the fit
LLfit.report(asymmetric_parameter_errors=True)

pLL = Plot(LLfit)  # create a plot object
pLL.x_range = [a, b]
pLL.plot(fit_info=True, asymmetric_parameter_errors=True)  # plot the data and the fit
#pLL.axes[0]['main'].set_xlabel('Life time '+r'$\tau$'+' (µs)', size='large')  # overwrite the x-axis label

cpfLL = ContoursProfiler(LLfit, profile_subtract_min=False)  # Optional: create a contours profile
cpfLL.plot_profiles_contours_matrix(parameters=['tau', 'fbg'])  # Optional: plot the contour matrix for tau and fbg

cpfLL.show()  # show the plot(s)
```

Of particular interest is the special mode of graphical representation of the data,
where each measured value is represented by a line.
The density of the lines per unit length corresponds to the distribution density.

In [ ]:
''' the data for the myon life time example'''
# real data from measurement with a Water Cherenkov detector ("Kamiokanne")
dT = [7.42, 3.773, 5.968, 4.924,  1.468,  4.664,  1.745,  2.144,  3.836,  3.132,
  1.568,  2.352,  2.132,  9.381,  1.484,  1.181,  5.004,  3.06,   4.582,  2.076,
  1.88,   1.337,  3.092,  2.265,  1.208,  2.753,  4.457,  3.499,  8.192,  5.101,
  1.572,  5.152,  4.181,  3.52,   1.344, 10.29,   1.152,  2.348,  2.228,  2.172,
  7.448,  1.108,  4.344,  2.042,  5.088,  1.02,   1.051,  1.987,  1.935,  3.773,
  4.092,  1.628,  1.688,  4.502,  4.687,  6.755,  2.56,   1.208,  2.649,  1.012,
  1.73,   2.164,  1.728,  4.646,  2.916,  1.101,  2.54,   1.02,   1.176,  4.716,
  9.671,  1.692,  9.292, 10.72,   2.164,  2.084,  2.616,  1.584,  5.236,  3.663,
  3.624,  1.051,  1.544,  1.496,  1.883,  1.92,   5.968,  5.89,   2.896,  2.76,
  1.475,  2.644,  3.6,    5.324,  8.361,  3.052,  7.703,  3.83,   1.444,  1.343,
  4.736,  8.7,    6.192,  5.796,  1.4,    3.392,  7.808,  6.344,  1.884,  2.332,
  1.76,   4.344,  2.988,  7.44,   5.804,  9.5,    9.904,  3.196,  3.012,  6.056,
  6.328,  9.064,  3.068,  9.352,  1.936,  1.08,   1.984,  1.792,  9.384, 10.15,
  4.756,  1.52,   3.912,  1.712, 10.57,   5.304,  2.968,  9.632,  7.116, 1.212,
  8.532,  3.000,  4.792,  2.512,  1.352,  2.168,  4.344,  1.316,  1.468, 1.152,
  6.024,  3.272,  4.96,  10.16,   2.14,   2.856, 10.01,   1.232, 2.668, 9.176 ]

In [ ]:
# try the likelihood fit here
# -->


***
## 8. Multi-Fits:
### simultaneous fit of model functions to different data sets
***
Very often the models are too complex to determine all parameters in a fit to a single model.
Instead model parameters are frequently the result of several fits,
or the same parameter occurs in different measurement series.

For such cases *kafe2* offers the possibility to perform several fits of different models with
common parameters to different data sets.

For this purpose, the *MultiFit* package must also be imported:
``` python
from kafe2 import MultiFit
```

As a simple example, we consider the determination of an ohmic resistance at room temperature,
which heats up at higher current flow and thus changes its resistance according to its
temperature coefficient.
Therefore, in addition to the current through the resistor, the temperature is measured for each
given voltage value.
Triplets of measured values must therefore be evaluated.

The temperature dependence is described empirically by a simple quadratic model:
``` python
# empirical model for T(U): a parabola
def empirical_T_U_model(U, p2=1.0, p1=1.0, p0=0.0):
    # use quadratic model as empirical temperature dependence T(U)
    return p2 * U**2 + p1 * U + p0
```

The resistance as a function of temperature is given by the temperature coefficient $\alpha$ and
is modeled as follows
``` python
# model of current-voltage dependence I(U) for a heating resistor
def I_U_model(U, R0=1., alph=0.004, p2=1.0, p1=1.0, p0=0.0):
    # use quadratic model as empirical temperature dependence T(U)
    t_ref = 0.
    _delta_t = empirical_T_U_model(U, p2, p1, p0) - t_ref
    # plug the temperature into the model
    return U / (R0 * (1.0 + _delta_t * alph))
```

So in this case the model for resistance contains the first model for the dependence of
temperature on the current determined by the applied voltage.

Here is the data for this example:
``` python
# the data
U = [ 0.5, 1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0, 4.5, 5.0, 5.5,
      6.0, 6.5, 7.0, 7.5, 8.0, 8.5, 9.0, 9.5, 10.0 ]
I = [ 0.5,  0.89, 1.41, 1.67, 2.3,  2.59, 2.77, 3.57, 3.94,  4.24, 4.73,
      4.87, 5.35, 5.74, 5.77, 6.17, 6.32, 6.83, 6.87, 7.17 ]
T = [ 20.35, 20.65, 22.25, 23.65, 26.25, 27.85, 29.85, 34.25, 37.75, 41.95,
     44.85, 50.05, 54.25, 60.55, 65.05, 69.95, 76.85, 81.55, 85.45, 94.75 ]
sigU, sigI, sigT = 0.2, 0.1, 0.5 # uncertainties
```

The Fit procedure is hardly different from the procedure presented so far.
First, the data containers and fits for the two models are defined as:
``` python
# Step 1: construct the singular data containers and fit objects
TU_data = XYContainer(U,T)
TU_data.label = 'Temperature data'
TU_data.axis_labels = ['Voltage (V)','Temperature (°C)']
fit_1 = Fit(TU_data, model_function=empirical_T_U_model)
fit_1.model_label = 'Parametrization'

IU_data = XYContainer(U,I)
IU_data.label = 'Current data'
IU_data.axis_labels = ['Voltage (V)','Current (A)']
fit_2 = Fit(IU_data, model_function=I_U_model)
fit_2.model_label = 'Temperature-dependent conductance'

```

Then both fits are combined to a *MultiFit*.
``` python
# Step 2: construct a MultiFit object
multi_fit = MultiFit(fit_list=[fit_1, fit_2], minimizer='iminuit')
```
Only now are the uncertainties added - this time to the fit objects.
This also allows the uncertainties on the x-axis, which are common to both data sets, to be taken
into account.
``` python
# Step 3: Add errors (to the fit object in this case)
multi_fit.add_error(sigT, 0, axis='y')  # declare errors on T
multi_fit.add_error(sigI, 1, axis='y')  # declare errors on I
multi_fit.add_error(sigU, 'all', axis='x') # shared error on x axis
```

The next step is to define meaningful names for the output:
``` python
# (Optional): assign names for models and parameters
multi_fit.assign_parameter_latex_names(
    U='U', p2='p_2', p1='p_1', p0='p_0', R0='R_0', alph=r'\alpha_\mathrm{T}')
multi_fit.assign_model_function_expression('{p2}*{U}^2 + {p1}*{U} + {p0}', fit_index=0)
multi_fit.assign_model_function_latex_expression(r'{p2}\,{U}^2 + {p1}\,{U} + {p0}', fit_index=0)
multi_fit.assign_model_function_expression('{U} / ({R0} * (1 + ({p2}*{U}^2 + {p1}*{U} + {p0}) * {alph}))', fit_index=1)
multi_fit.assign_model_function_latex_expression(r'\frac{{{U}}}{{{R0} \cdot (1 + ({p2}{U}^2 + {p1}{U} + {p0}) \cdot {alph})}}', fit_index=1)
```

The rest works the same way as before:
``` python
# Step 4: do the fit
multi_fit.do_fit()

# (Optional): print the results
multi_fit.report()

# (Optional): plot the results
plot = Plot(multi_fit, separate_figures=True)
plot.customize('data', 'marker', ['.','.'])
plot.customize('data', 'markersize', [6,6])

plot.plot()

plot.show()
```

In [ ]:
# enter own code here
# -->
